In [5]:
from typing import cast

from cfrpy.pkr.game import RiverOfBlood, Fold, Call
from cfrpy.pkr.abstraction.versions.v1 import Abstraction
from cfrpy.algorithms.algo import Runner
from cfrpy.algorithms.cfrplus import ESCFRP
from cfrpy.algorithms.escfr import ESCFR
from cfrpy.algorithms.lcfr import ESLCFR
from cfrpy.game import Player, mc, play

In [6]:
g = Abstraction()
g = g.apply(g.sample())
g = g.apply(g.sample())
g = g.apply(Call()).apply(Call())
g = g.apply(g.sample())

g.infoset(g.active).actions()

(Check(), Bet(bet=8), Bet(bet=16), Bet(bet=40), Bet(bet=80), Bet(bet=120))

In [7]:
runner = Runner(ESLCFR(10_000), Abstraction, "checkpt{}.pkl", 10000, None)

In [8]:
runner.run()

2023-01-22 16:57:23 [info     ] iteration                      infosets=131 it=0 touched=243
2023-01-22 16:57:25 [info     ] iteration                      infosets=1158 it=10 touched=3023
2023-01-22 16:57:27 [info     ] iteration                      infosets=1661 it=20 touched=5632
2023-01-22 16:57:29 [info     ] iteration                      infosets=2016 it=30 touched=8206
2023-01-22 16:57:32 [info     ] iteration                      infosets=2365 it=40 touched=11248
2023-01-22 16:57:34 [info     ] iteration                      infosets=2606 it=50 touched=13829
2023-01-22 16:57:36 [info     ] iteration                      infosets=2821 it=60 touched=16807
2023-01-22 16:57:38 [info     ] iteration                      infosets=2981 it=70 touched=19086
2023-01-22 16:57:40 [info     ] iteration                      infosets=3117 it=80 touched=21797
2023-01-22 16:57:43 [info     ] iteration                      infosets=3314 it=90 touched=24651
2023-01-22 16:57:45 [info     ] itera

KeyboardInterrupt: 